# Data preprocessing
We have seen that the users that spends the most (in total) are the ones that buy a lot of products with average prices. At the same time, the products more profitable have the same features, lots of purchases and an average price.

In [1]:
# Loading useful libraries

import numpy as np
import pandas as pd

# import optuna
# import random

# import matplotlib.pyplot as plt
# import seaborn as sns

# from copy import deepcopy
# from lightgbm import LGBMClassifier, LGBMRegressor
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# from sklearn.metrics import accuracy_score, f1_score, PrecisionRecallDisplay, precision_score, recall_score, r2_score
# from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# Getting dataset from CS

# data = pd.read_csv('gs://engo-ml_spec2023-demo2/data_raw.csv')
# data.head()

In [3]:
# Getting dataset from local file

users = pd.read_csv('../data/tmp/users_train.csv', index_col='User_ID')
users_test = pd.read_csv('../data/tmp/users_test.csv', index_col='User_ID')

users.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Number of Purchases,Sum spent,Average Purchase
User_ID,,,,,,,,,
1003663,F,36-45,9,B,1,0,45,549674,12214.977778
1001386,F,26-35,0,B,1,0,42,385080,9168.571429
1005344,F,51-55,1,C,3,0,121,953172,7877.454545
1000298,F,18-25,4,C,2,0,15,137272,9151.466667
1004679,M,55+,13,C,4+,1,37,280273,7574.945946


In [4]:
# seed = 42

# users=pd.DataFrame()
# users['Gender'] = data.groupby('User_ID').agg({'Gender': 'first'})
# users['Age'] = data.groupby('User_ID').agg({'Age': 'first'})
# users['Occupation'] = data.groupby('User_ID').agg({'Occupation': 'first'})
# users['City_Category'] = data.groupby('User_ID').agg({'City_Category': 'first'})
# users['Stay_In_Current_City_Years'] = data.groupby('User_ID').agg({'Stay_In_Current_City_Years': 'first'})
# users['Marital_Status'] = data.groupby('User_ID').agg({'Marital_Status': 'first'})
# users['Number of Purchases'] = data.groupby('User_ID').size()
# users['Sum spent'] = data.groupby('User_ID')['Purchase'].sum()
# users['Average Purchase'] = data.groupby('User_ID')['Purchase'].mean()

# users_train, users_test = train_test_split(users, test_size=.2, random_state=seed, stratify=users[['Gender', 'Age', 'City_Category', 'Marital_Status']])

# df = deepcopy(data)[data['User_ID'].isin(users_train.index)]
# df_test = deepcopy(data)[data['User_ID'].isin(users_test.index)]

In [5]:
# # Grouping by user ("User_ID")

# users = pd.DataFrame()
# users['Gender'] = df.groupby('User_ID').agg({'Gender': 'first'})
# users['Age'] = df.groupby('User_ID').agg({'Age': 'first'})
# users['Occupation'] = df.groupby('User_ID').agg({'Occupation': 'first'})
# users['City_Category'] = df.groupby('User_ID').agg({'City_Category': 'first'})
# users['Stay_In_Current_City_Years'] = df.groupby('User_ID').agg({'Stay_In_Current_City_Years': 'first'})
# users['Marital_Status'] = df.groupby('User_ID').agg({'Marital_Status': 'first'})
# users['Number of Purchases'] = df.groupby('User_ID').size()
# users['Sum spent'] = df.groupby('User_ID')['Purchase'].sum()
# users['Average Purchase'] = df.groupby('User_ID')['Purchase'].mean()
# users=users.sort_values(by='Sum spent', ascending=False)
# users

In [6]:
# Dividing "big-spenders" from the rest

threshold = np.quantile(users['Sum spent'], 0.70, axis=0)
print(f'The big-spender threshold is set at {threshold}, that is the 0.9 quantile for "Sum spent".')

users['is_BigSpender'] = (users['Sum spent'] >= threshold).astype(int)
users

The big-spender threshold is set at 946918.7, that is the 0.9 quantile for "Sum spent".


,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Number of Purchases,Sum spent,Average Purchase,is_BigSpender
User_ID,,,,,,,,,,
1003663,F,36-45,9,B,1,0,45,549674,12214.977778,0
1001386,F,26-35,0,B,1,0,42,385080,9168.571429,0
1005344,F,51-55,1,C,3,0,121,953172,7877.454545,1
1000298,F,18-25,4,C,2,0,15,137272,9151.466667,0
1004679,M,55+,13,C,4+,1,37,280273,7574.945946,0
...,...,...,...,...,...,...,...,...,...,...
1001288,F,36-45,19,C,1,0,80,606034,7575.425000,0
1004131,M,46-50,7,C,2,1,70,722849,10326.414286,0
1004524,M,26-35,2,A,2,1,150,1379212,9194.746667,1


In [7]:
# Creating test set

# users_test = pd.DataFrame()
# users_test['Gender'] = df_test.groupby('User_ID').agg({'Gender': 'first'})
# users_test['Age'] = df_test.groupby('User_ID').agg({'Age': 'first'})
# users_test['Occupation'] = df_test.groupby('User_ID').agg({'Occupation': 'first'})
# users_test['City_Category'] = df_test.groupby('User_ID').agg({'City_Category': 'first'})
# users_test['Stay_In_Current_City_Years'] = df_test.groupby('User_ID').agg({'Stay_In_Current_City_Years': 'first'})
# users_test['Marital_Status'] = df_test.groupby('User_ID').agg({'Marital_Status': 'first'})
# users_test['Number of Purchases'] = df_test.groupby('User_ID').size()
# users_test['Sum spent'] = df_test.groupby('User_ID')['Purchase'].sum()
# users_test['Average Purchase'] = df_test.groupby('User_ID')['Purchase'].mean()
users_test['is_BigSpender'] = (users_test['Sum spent'] >= threshold).astype(int)
# users_test=users_test.sort_values(by='Sum spent', ascending=False)
users_test

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Number of Purchases,Sum spent,Average Purchase,is_BigSpender
User_ID,,,,,,,,,,
1005452,M,36-45,6,A,4+,0,162,1485500,9169.753086,1
1005543,M,26-35,17,A,2,0,190,1891121,9953.268421,1
1005881,M,36-45,17,B,1,0,159,1415078,8899.861635,1
1001531,M,46-50,17,C,2,1,19,144107,7584.578947,0
1004180,M,55+,13,C,1,1,54,531531,9843.166667,0
...,...,...,...,...,...,...,...,...,...,...
1002041,F,26-35,20,B,4+,0,312,2499908,8012.525641,1
1003846,M,36-45,0,C,0,0,36,302041,8390.027778,0
1004301,M,36-45,11,C,1,1,18,198664,11036.888889,0


In [8]:
# Selecting "regression" or "classification"

model_type = "classification"

assert(model_type in ["regression", "classification"])

In [9]:
# Dropping useless columns

users.drop(['Number of Purchases', 'Average Purchase'], axis=1, inplace=True)
users_test.drop(['Number of Purchases', 'Average Purchase'], axis=1, inplace=True)

if model_type == "regression":
    users.drop(['is_BigSpender'], axis=1, inplace=True)
    users_test.drop(['is_BigSpender'], axis=1, inplace=True)
    y_col = 'Sum spent'
else:
    users.drop(['Sum spent'], axis=1, inplace=True)
    users_test.drop(['Sum spent'], axis=1, inplace=True)
    y_col = 'is_BigSpender'

In [10]:
# Separating "X" and "y"

X_train = users.drop(y_col, axis=1)
y_train = users[y_col]

X_test = users_test.drop(y_col, axis=1)
y_test = users_test[y_col]

In [11]:
# Encoding categorical columns

for column in X_train.columns:
    if X_train[column].dtype == 'object':
        label_encoder = LabelEncoder()
        X_train[column] = label_encoder.fit_transform(X_train[column])
        X_test[column] = label_encoder.transform(X_test[column])

# Saving data

In [12]:
X_train.to_csv(f'../data/tmp/X_train_{model_type}.csv')
y_train.to_csv(f'../data/tmp/y_train_{model_type}.csv')
X_test.to_csv(f'../data/tmp/X_test_{model_type}.csv')
y_test.to_csv(f'../data/tmp/y_test_{model_type}.csv')